In [59]:
import deepcut
from pathlib import Path
from pythainlp import word_tokenize
from matplotlib import pyplot as plt

import re
import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from nltk import FreqDist, precision, recall, f_measure, NaiveBayesClassifier
from nltk.classify import apply_features
from nltk.classify import util
import collections, itertools
from sklearn.model_selection import KFold

In [60]:
data = pd.read_csv('train.csv')

In [ ]:
'''
texts = []
labels = []
with open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train.txt",encoding="utf8") as f:
    for line in f:
        texts.append(line.strip())
        
with open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train_label.txt",encoding="utf8") as f:
    for line in f:
        labels.append(line.strip())

df = pd.DataFrame({"category":labels, "texts":texts})
df.shape
'''

In [61]:
data = data[data.labels != 'q']

In [62]:
data.groupby('labels').describe()

words                                                               
        count unique                                                top freq
labels                                                                      
neg      7393   7393  แจกจ้าครบรอบทั้งทีเพื่อพี่น้องยอมเสียตัง555 ปร...    1
neu     13087  13087  เบียร์ระดมกิมมิกแรงส์ปลุกยอด ไฮเนเก้นรุกสายปาร...    1
pos      4617   4617  ฮอนด้า ซีวิค สวยมากเลยคับ อยากได้สักคัน สวยหรู...    1

In [63]:
data['length'] = data['words'].apply(word_tokenize).apply(len)
data['length'].describe()

count    25097.000000
mean        25.850779
std         44.237998
min          1.000000
25%          5.000000
50%         11.000000
75%         28.000000
max        758.000000
Name: length, dtype: float64

In [64]:
data

,words,labels,length
0,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ,neu,13
1,คะ,neu,1
2,อิเหี้ยออมทำกูอยากกินเอ็มเค,neg,9
3,😅😅😅,neu,1
4,สวัสดีวันพุธ แนน อะไรนะ,neu,8
...,...,...,...
25610,อีชาติชั้ว,neg,3
25611,อีช้างเย๊ด,neg,3
25612,อยากเอาหญิง,neg,3
25613,ทมิฬ,neg,1


In [65]:
data.sample(100)

,words,labels,length
13911,มั่นใจกับการขับขี่ยามค่ำคืน ที่คุณไร้กังวลการใ...,neg,38
21552,คือรูปที่อยู่ในกรอบสีม่วงคันที่ 4 นับจากซ้ายไป...,neu,31
16479,ไม่ไปล่ะ,neu,3
7195,เรามาแก้ไข ระดับ EQ ในการทำงานกันเถอะ #เรื่องส...,neu,26
5652,ป๊า กิน Leo แต่ ยูโร ดื่ม Pepsi ครับ 😄😄😄,neu,17
...,...,...,...
8034,เรื่องราวของนักศึกษาคนหนึ่งที่สอบยังไงคะแนนก็ไ...,neu,62
10676,จะได้กินไหมห๊า,pos,5
5159,มาสด้าเก๋งยังหน้าเหมือนกันทุกรุ่น ถ้าไม่ใช่แฟน...,neu,21
11017,เนตทรูหลุดบ่อย เดือนนี้มาซ่อมแล้ว 3 ครั้ง ก็ยั...,neg,20


In [66]:
from gensim.models import Word2Vec 

In [67]:
y_train = [row for row in data['labels']]
Y_train = array(y_train)
Y_train.shape

(25097,)

In [68]:
label_encoder = LabelEncoder()

In [69]:
integer_encoder = label_encoder.fit_transform(Y_train)

In [70]:
integer_encoder

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [71]:
ohe = OneHotEncoder(sparse=False)

In [72]:
onehot_encoded = ohe.fit_transform(integer_encoder.reshape(-1,1))

In [73]:
onehot_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [74]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.words.values)

In [75]:
word_index = tokenizer.word_index

In [76]:
vocab_size = len(word_index)

In [77]:
vocab_size

87090

In [78]:
data['length'].describe()

count    25097.000000
mean        25.850779
std         44.237998
min          1.000000
25%          5.000000
50%         11.000000
75%         28.000000
max        758.000000
Name: length, dtype: float64

In [79]:
max_length = 758

In [80]:
x_train = tokenizer.texts_to_sequences(data.words.values)

In [81]:
x_train_padded = pad_sequences(x_train, maxlen=max_length ,padding='post')

In [82]:
x_train_padded.shape

(25097, 758)

In [83]:
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout, Activation
from tensorflow.keras.models import Sequential

In [84]:
model = Sequential()
model.add(Embedding(vocab_size+1,32,input_length=max_length))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 758, 32)           2786912   
_________________________________________________________________
flatten_2 (Flatten)          (None, 24256)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               3104896   
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
Total params: 5,892,195
Trainable params: 5,892,195
Non-trainable params: 0
_________________________________________________________________


In [85]:
from sklearn.model_selection import train_test_split

In [86]:
x_train, x_test, y_train, y_test = train_test_split(x_train_padded, onehot_encoded, test_size=0.3, shuffle=True)

In [87]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((17567, 758), (17567, 3), (7530, 758), (7530, 3))

In [ ]:
model.fit(x_train,y_train,
         batch_size=20,
         epochs=5,
         verbose=1,
         validation_data=(x_test,y_test))

Train on 17567 samples, validate on 7530 samples
Epoch 1/5
17567/17567 [==============================] - 47s 3ms/sample - loss: 0.0131 - accuracy: 0.9969 - val_loss: 1.3044 - val_accuracy: 0.4968
Epoch 2/5
17567/17567 [==============================] - 51s 3ms/sample - loss: 0.0094 - accuracy: 0.9973 - val_loss: 1.3959 - val_accuracy: 0.5104
Epoch 3/5
17567/17567 [==============================] - 43s 2ms/sample - loss: 0.0078 - accuracy: 0.9979 - val_loss: 1.5068 - val_accuracy: 0.5116
Epoch 4/5
17567/17567 [==============================] - 49s 3ms/sample - loss: 0.0079 - accuracy: 0.9979 - val_loss: 1.7014 - val_accuracy: 0.5102
Epoch 5/5
12320/17567 [====================>.........] - ETA: 15s - loss: 0.0042 - accuracy: 0.9989- 

In [ ]:
test_loss,test_acc = model.evaluate(x_test,y_test)
print("\nTest accuracy:",test_acc)

In [91]:
predicions = model.predict(x_test)